In [1]:
from osgeo import ogr
import shapely, fiona, json

In [4]:
raw_strings = sc.textFile("result_data/all_tweets_one_file.jsonl") # Would Using HDFS be smarter here?

In [5]:
raw_strings.take(1)

[u'{"geometry": {"type": "Point", "coordinates": [-74.5342206, 39.3954102]}, "type": "Feature", "properties": {"u_utc": null, "text": "- I do my own thing , you made me this way .", "source": {"link": "http://twitter.com/download/android", "displayName": "Twitter for Android"}, "verb": "post", "user": "Dougieee_Nastyy", "time": "2012-10-25T11:25:26.000Z", "gnip": {"matching_rules": [{"tag": "[-74.53321912339669,39.36436374931048]", "value": "bounding_box:[-74.57813488760267 39.329638110038545 -74.48830335919072 39.39908938858241]"}], "language": {"value": "en"}}}}']

In [6]:
tweets = raw_strings.map(json.loads)
tweets.cache()

PythonRDD[5] at RDD at PythonRDD.scala:53

In [7]:
t1 = tweets.take(1)[0]

In [8]:
print(t1)

{u'geometry': {u'type': u'Point', u'coordinates': [-74.5342206, 39.3954102]}, u'type': u'Feature', u'properties': {u'u_utc': None, u'text': u'- I do my own thing , you made me this way .', u'source': {u'displayName': u'Twitter for Android', u'link': u'http://twitter.com/download/android'}, u'verb': u'post', u'user': u'Dougieee_Nastyy', u'time': u'2012-10-25T11:25:26.000Z', u'gnip': {u'matching_rules': [{u'tag': u'[-74.53321912339669,39.36436374931048]', u'value': u'bounding_box:[-74.57813488760267 39.329638110038545 -74.48830335919072 39.39908938858241]'}], u'language': {u'value': u'en'}}}}


## Load Zone A as a Geometry

In [9]:
from shapely.geometry import mapping, shape
c = fiona.open('ZoneA_Geometry/ZoneA/OGRGeoJSON.shp','r')
pol = c.next()
geom = shape(pol['geometry']).buffer(0)
geom.is_valid

True

In [10]:
zoneABroadcast = sc.broadcast(geom)

In [1]:
test_tweets = tweets_with_geo.take(10)

NameError: name 'tweets_with_geo' is not defined

In [11]:
#Testing:
in_bounds = shape({'type': "Point", 'coordinates': [-73.94900, 40.73389]})
print in_bounds
print( geom.contains(in_bounds))
# print("In Bounds (Should be True): " + str( point_in_bounds.Within(zoneABroadcast.value)))

out_of_bounds = shape({"type": "Point","coordinates": [-73.99753114562988,40.73093368341445]})
print out_of_bounds
print geom.contains(out_of_bounds)

POINT (-73.949 40.73389)
True
POINT (-73.99753114562988 40.73093368341445)
False


In [12]:
def get_point(tweet):
    tweet['geo']['coordinates'].reverse()
    return shape(tweet['geo'])

#Test it:
for t in test_tweets:
p = get_point(t1)
print(p)
p

POINT (-74.5342206 39.3954102)


### Use Spark to find ZoneA tweets

In [13]:
inZoneA = tweets.filter(lambda t: zoneABroadcast.value.contains(get_point(t)))

In [14]:
# Kick off the Action, save the tweets
inZoneA.cache()
inZoneA.count()

110111

In [16]:
inZoneAStrings = inZoneA.map(json.dumps)

In [17]:
inZoneAStrings.saveAsTextFile('inZoneA.geojsonl.hdfs')

In [18]:
grouped_by_user = inZoneA.groupBy(lambda t: t['properties']['user'])
count_by_user   = grouped_by_user.map(lambda tu: (tu[0],len(tu[1]) ))
sorted_by_quant = count_by_user.sortBy(lambda tu: -tu[1])
sorted_by_quant.cache()
print("Users: " + str(sorted_by_quant.count()))

Users: 22042


In [ ]:
# Write out all the tweets to GeoJSON

# Now build a secondary GeoJSON server to only serve specific dates?

# for tu in sorted_by_quant.collect():
#     u_tweets = tweets.filter(lambda t: t['properties']['user'] == tu[0])
#     features = u_tweets.collect()
#     with open('zone_a_twitterers/'+tu[0]+'.geojson','w') as outGeo:
#         json.dump({"type":"FeatureCollection","features":features},outGeo)

# Temporal Filtering of Zone A Tweets

In [21]:
import json, datetime, iso8601, pytz

In [22]:
def between_two_dates(tweet, start, end):
        time = iso8601.parse_date(tweet['properties']['time'])
#         del tweet['properties']['gnip']
        return time > start and time < end

In [25]:
LANDFALL = datetime.datetime(2012,10,29,20,0,0,tzinfo=pytz.utc) #Landfall as UTC
AFTER    = datetime.datetime(2012,11,1,0,0,0,tzinfo=pytz.utc)   #After as UTC

inZoneA_landfall = inZoneA.filter( lambda tweet: between_two_dates(tweet, LANDFALL, AFTER) )

In [26]:
inZoneA_landfall.cache()
inZoneA_landfall.count()

2466

In [28]:
zoneALandfall_grouped_by_user = inZoneA_landfall.groupBy(lambda t: t['properties']['user'])
zoneALandfall_count_by_user   = zoneALandfall_grouped_by_user.map(lambda tu: (tu[0],len(tu[1]) ))
zoneALandfall_sorted_by_quant = zoneALandfall_count_by_user.sortBy(lambda tu: -tu[1])
zoneALandfall_sorted_by_quant.cache()
print("Users: " + str(zoneALandfall_sorted_by_quant.count()))

Users: 916


In [29]:
zoneALandfall_sorted_by_quant.take(100)

[(u'Zav718', 106),
 (u'hermaflease', 90),
 (u'jennifferkkwilt', 90),
 (u'charlenexcfye', 86),
 (u'innycru', 56),
 (u'mr0drig', 55),
 (u'laynaBrady', 33),
 (u'honeyberk', 31),
 (u'lucida_console', 25),
 (u'carminenyc', 23),
 (u'Omg_itsClyde', 20),
 (u'JenniferCochra1', 17),
 (u'JoeyBoots', 17),
 (u'laurakazam', 17),
 (u'salvadorsorian5', 16),
 (u'Sarahphara', 15),
 (u'GeorgieeeNinjaa', 14),
 (u'anneeoanneo', 14),
 (u'frankgiraldi', 14),
 (u'BobSpivak1', 13),
 (u'mcflipper', 13),
 (u'fefaromano', 13),
 (u'epjohanson', 13),
 (u'eelain212', 12),
 (u'rahav', 11),
 (u'JUSTmeALISEO', 10),
 (u'johnjohnta', 10),
 (u'AdamGabbatt', 10),
 (u'kirilnyc', 9),
 (u'cpthady', 9),
 (u'j_livoti', 9),
 (u'mmmpork', 9),
 (u'M3lissaSays', 9),
 (u'bgallis', 9),
 (u'AnthonyQuintano', 9),
 (u'JayDensonNYC', 9),
 (u'DaReal_JrJones', 9),
 (u'smoshysydney', 9),
 (u'julienneschaer', 8),
 (u'JonMcL', 8),
 (u'JamieAlexandraa', 8),
 (u'marky_mark34', 8),
 (u'taftcard', 8),
 (u'LucaTrippitelli', 8),
 (u'MrsPuertoRico',